In [1]:
#!pip install -U langchain-openai
#!pip install python-dotenv

In [2]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_TRACING = os.getenv('LANGSMITH_TRACING')

# ChatOpenAI

In [5]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2)

In [6]:
messages = [
           ("system","You are a helpful assistant that translates English to French. Translate the user sentence."),
           ("human", "I love programming."),
           ]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 31, 'total_tokens': 36}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d576307f90', 'finish_reason': 'stop', 'logprobs': None}, id='run-9243114a-bb08-4554-9307-a653d169a7bf-0', usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36})

In [7]:
print(ai_msg.content)

J'adore la programmation.


# Chaining

In [8]:
from langchain_core.prompts import ChatPromptTemplate

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 26, 'total_tokens': 32}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d576307f90', 'finish_reason': 'stop', 'logprobs': None}, id='run-acf69d85-0d0f-4921-97ac-d0f37e60d102-0', usage_metadata={'input_tokens': 26, 'output_tokens': 6, 'total_tokens': 32})

# Tool Calling

#### ChatOpenAI.bind_tools()

Lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool. tool-calling is extremely useful for building tool-using chains and agents, and for getting structured outputs from models more generally.

In [10]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [11]:
class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

In [12]:
llm_with_tools = llm.bind_tools([GetWeather])

In [13]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in Long Grove",
)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TskRR5wPU7FCUmtPEXJTiY6r', 'function': {'arguments': '{"location":"Long Grove, IL"}', 'name': 'GetWeather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 68, 'total_tokens': 85}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ce0793330f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-54b583f6-b7e0-4f5a-aae2-a49105db01aa-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'Long Grove, IL'}, 'id': 'call_TskRR5wPU7FCUmtPEXJTiY6r'}], usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85})

In [14]:
ai_msg.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'Long Grove, IL'},
  'id': 'call_TskRR5wPU7FCUmtPEXJTiY6r'}]

In [15]:
ai_msg.content

''

# Fine-tuning

Fine tuned models should be in the form of ft:{OPENAI_MODEL_NAME}:{ORG_NAME}::{MODEL_ID}. 

In [16]:
#fine_tuned_model = ChatOpenAI(
#    temperature=0, model_name="ft:gpt-3.5-turbo-0613:langchain::7qTVM5AR"
#)

#fine_tuned_model(messages)